In [8]:
import numpy as np
from scipy.optimize import minimize

# Constants
g = 9.81  # Gravity (m/s^2)

# Function to calculate the weight transfer
def weight_transfer(deceleration, weight, cg_height, wheelbase):
    return (deceleration * weight * cg_height) / wheelbase

# Function to calculate available grip
def available_grip(weight_distribution, weight_transfer, weight, mu_tire):
    grip_front = (weight * weight_distribution + weight_transfer) * mu_tire
    grip_rear = (weight * (1 - weight_distribution) - weight_transfer) * mu_tire
    return grip_front, grip_rear

# Objective function to maximize deceleration (minimize negative deceleration)
def objective_function(x, weight, weight_distribution, cg_height, wheelbase, mu_tire):
    mass, mu, caliper_area_front, caliper_area_rear, mc_area_front, mc_area_rear, bias, pedal_force, pedal_ratio, tire_radius = x
    
    pressure_front = (pedal_force * pedal_ratio) / mc_area_front
    pressure_rear = (pedal_force * pedal_ratio) / mc_area_rear
    
    braking_force_front = mu * caliper_area_front * pressure_front
    braking_force_rear = mu * caliper_area_rear * pressure_rear
    
    deceleration = (braking_force_front + braking_force_rear) / mass
    wt = weight_transfer(deceleration * g, weight, cg_height, wheelbase)
    
    grip_limit_front, grip_limit_rear = available_grip(weight_distribution, wt, weight, mu_tire)
    
    effective_braking_force_front = min(braking_force_front, grip_limit_front)
    effective_braking_force_rear = min(braking_force_rear, grip_limit_rear)
    
    total_effective_braking_force = effective_braking_force_front + effective_braking_force_rear
    
    decel = total_effective_braking_force / mass
    decel_g = decel / g
    
    return -decel_g

# Bounds for the parameters
bounds = [
    (250, 300),  # Mass (kg)
    (0.35, 0.55),  # Brake pad mu
    (0.0005, 0.005),  # Caliper piston area front (m^2)
    (0.0005, 0.005),  # Caliper piston area rear (m^2)
    (0.00005, 0.0005),  # Master cylinder bore area front (m^2)
    (0.00005, 0.0005),  # Master cylinder bore area rear (m^2)
    (0.3, 0.7),  # Brake bias
    (200, 700),  # Pedal force (N)
    (4, 6),  # Pedal ratio
    (0.2, 0.4),  # Tire radius (m)
]

# Initial guess for the parameters
initial_guess = [
    275,  # Mass (kg)
    0.45,  # Brake pad mu
    0.002,  # Caliper piston area front (m^2)
    0.002,  # Caliper piston area rear (m^2)
    0.0002,  # Master cylinder bore area front (m^2)
    0.0002,  # Master cylinder bore area rear (m^2)
    0.6,  # Brake bias
    300,  # Pedal force (N)
    5,  # Pedal ratio
    0.3,  # Tire radius (m)
]

# Given vehicle parameters
weight = 2712.6  # Total weight (N)
weight_distribution = 0.505  # Percentage of weight on the front axle
cg_height = 0.267462  # Center of gravity height (m)
wheelbase = 1.524  # Wheelbase (m)
mu_tire = 1.0  # Coefficient of friction between tires and road

# Optimization
result = minimize(
    objective_function,
    initial_guess,
    args=(weight, weight_distribution, cg_height, wheelbase, mu_tire),
    bounds=bounds
)

if result.success:
    optimized_parameters = result.x
    max_decel_g = -result.fun
    parameter_names = [
        "Total mass (kg)",
        "Brake pad mu",
        "Caliper piston area front (m^2)",
        "Caliper piston area rear (m^2)",
        "Master cylinder bore area front (m^2)",
        "Master cylinder bore area rear (m^2)",
        "Brake bias",
        "Pedal force (N)",
        "Pedal ratio",
        "Tire radius (m)"
    ]
    print("Optimized parameters:")
    for name, param in zip(parameter_names, optimized_parameters):
        print(f"{name}: {param:.4f}")
    print(f"Max deceleration (g's): {max_decel_g:.4f}")
else:
    raise ValueError(result.message)


Optimized parameters:
Total mass (kg): 300.0000
Brake pad mu: 0.3500
Caliper piston area front (m^2): 0.0005
Caliper piston area rear (m^2): 0.0005
Master cylinder bore area front (m^2): 0.0005
Master cylinder bore area rear (m^2): 0.0005
Brake bias: 0.6000
Pedal force (N): 200.0000
Pedal ratio: 4.0000
Tire radius (m): 0.3000
Max deceleration (g's): -2.4108
